In [1]:
import re
import kss
import unicodedata
f = open('4BH00005.txt', encoding="utf8")
i=0
sentence_list = []
while True:
    line = f.readline()
    if not line: break
    if line != '\n':
        i=i+1
        line = re.sub('<.+?>', '', line, 0, re.S)
        line = re.sub('\(.+?\)', '', line, 0, re.S)
        line = re.sub('\[.+?\]', '', line, 0, re.S)
        line = re.sub('\{.+?\}', '', line, 0, re.S)
        line = re.sub('《.+?》', '', line, 0, re.S)
        line = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!『』\‘|\(\)\[\]\<\>`\'…《》\{\}_「」±√]', '', line,0,re.S)
        line = line.lower()
        if len(line) >10:
            sentence_list+= kss.split_sentences(line)
f.close()
print(len(sentence_list))

2264


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentence_list)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 13809


In [3]:
sequences = list()

for line in sentence_list:
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[4919, 232],
 [4919, 232, 25],
 [4919, 232, 25, 2797],
 [4919, 232, 25, 2797, 4920],
 [8, 609],
 [8, 609, 610],
 [8, 609, 610, 611],
 [8, 609, 610, 611, 215],
 [8, 609, 610, 611, 215, 1169],
 [8, 609, 610, 611, 215, 1169, 232],
 [8, 609, 610, 611, 215, 1169, 232, 1170]]

In [4]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 1번 단어 : 있다


In [5]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 90


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [7]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [8]:
print(X[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 4919]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 4919  232]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  

In [9]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(GRU(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Train on 38358 samples
Epoch 1/200
38358/38358 - 87s - loss: 9.0000 - accuracy: 0.0131
Epoch 2/200
38358/38358 - 86s - loss: 8.3728 - accuracy: 0.0169
Epoch 3/200
38358/38358 - 86s - loss: 7.9932 - accuracy: 0.0232
Epoch 4/200
38358/38358 - 87s - loss: 7.5183 - accuracy: 0.0306
Epoch 5/200
38358/38358 - 84s - loss: 6.8870 - accuracy: 0.0452
Epoch 6/200
38358/38358 - 84s - loss: 6.2084 - accuracy: 0.0683
Epoch 7/200
38358/38358 - 84s - loss: 5.5521 - accuracy: 0.1082
Epoch 8/200
38358/38358 - 84s - loss: 4.9367 - accuracy: 0.1786
Epoch 9/200
38358/38358 - 84s - loss: 4.3666 - accuracy: 0.2682
Epoch 10/200
38358/38358 - 85s - loss: 3.8425 - accuracy: 0.3570
Epoch 11/200
38358/38358 - 85s - loss: 3.3775 - accuracy: 0.4319
Epoch 12/200
38358/38358 - 85s - loss: 2.9667 - accuracy: 0.4984
Epoch 13/200
38358/38358 - 84s - loss: 2.6211 - accuracy: 0.5533
Epoch 14/200
38358/38358 - 86s - loss: 2.3250 - accuracy: 0.5988
Epoch 15/200
38358/38358 - 86s - loss: 2.0708 - accuracy: 0.6408
Epoch 16/20

Epoch 127/200
38358/38358 - 98s - loss: 0.2265 - accuracy: 0.9394
Epoch 128/200
38358/38358 - 96s - loss: 0.2088 - accuracy: 0.9439
Epoch 129/200
38358/38358 - 95s - loss: 0.1941 - accuracy: 0.9468
Epoch 130/200
38358/38358 - 95s - loss: 0.1908 - accuracy: 0.9481
Epoch 131/200
38358/38358 - 95s - loss: 0.2032 - accuracy: 0.9452
Epoch 132/200
38358/38358 - 96s - loss: 0.2107 - accuracy: 0.9427
Epoch 133/200
38358/38358 - 96s - loss: 0.2035 - accuracy: 0.9440
Epoch 134/200
38358/38358 - 96s - loss: 0.2025 - accuracy: 0.9449
Epoch 135/200
38358/38358 - 97s - loss: 0.1876 - accuracy: 0.9488
Epoch 136/200
38358/38358 - 97s - loss: 0.1991 - accuracy: 0.9467
Epoch 137/200
38358/38358 - 97s - loss: 0.2175 - accuracy: 0.9409
Epoch 138/200
38358/38358 - 96s - loss: 0.2179 - accuracy: 0.9402
Epoch 139/200
38358/38358 - 98s - loss: 0.2067 - accuracy: 0.9438
Epoch 140/200
38358/38358 - 93s - loss: 0.1923 - accuracy: 0.9473
Epoch 141/200
38358/38358 - 97s - loss: 0.2042 - accuracy: 0.9449
Epoch 142/

In [13]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=89, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [17]:
print(sentence_generation(model, t, '학교의', 4))

학교의 투명성 학사관리의 투명성과 신뢰성


In [18]:
model.save('model.h5')

In [20]:
from keras.models import load_model
model_new = load_model('model.h5')

Using TensorFlow backend.


ValueError: Unknown initializer: GlorotUniform